In [1]:
!pip install pyspark py4j

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.3 MB 53 kB/s 
     |████████████████████████████████| 200 kB 81.3 MB/s 
     |████████████████████████████████| 199 kB 62.6 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.0-py2.py3-none-any.whl size=281764026 sha256=273eb662131fe06685d647cc9eb349ba1b4fa7c3b2020f44805a51efb2e82c03
  Stored in directory: /root/.cache/pip/wheels/7a/8e/1b/f73a52650d2e5f337708d9f6a1750d451a7349a867f928b885
Successfully built pyspark


In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("test_pySpark").getOrCreate()

In [2]:
from pyspark import SparkContext
sc=SparkContext(master="local",appName="transformaciones y acciones")

In [ ]:
#creamos rdd
rddl = sc.parallelize([1,2,3])
type(rddl)

#para visualizar el rdd
rddl.collect()

[1, 2, 3]

In [3]:
path = "/content/filesSparkCourse/"


In [4]:

#toma el archivo como un gran string y corta cada linea donde 
#haya una coma
equiposOlimpicosRDD = sc.textFile(path+"paises.csv") \
.map(lambda line : line.split(","))

#que nos muestra 5 lineas del rdd
equiposOlimpicosRDD.take(8)

[['id', 'equipo', 'sigla'],
 ['1', '30. Februar', 'AUT'],
 ['2', 'A North American Team', 'MEX'],
 ['3', 'Acipactli', 'MEX'],
 ['4', 'Acturus', 'ARG'],
 ['5', 'Afghanistan', 'AFG'],
 ['6', 'Akatonbo', 'IRL'],
 ['7', 'Alain IV', 'SUI']]

In [ ]:
#revisa los paises que compiten en las olimpiadas
equiposOlimpicosRDD.map(lambda x: (x[2])).distinct().count()

231

In [ ]:
#agrupa numero de equipos por pais
equiposOlimpicosRDD.map(lambda x: (x[2],x[1])).groupByKey().mapValues(len).take(10)

[('sigla', 1),
 ('AUT', 11),
 ('MEX', 9),
 ('ARG', 18),
 ('AFG', 1),
 ('IRL', 7),
 ('SUI', 17),
 ('ALB', 1),
 ('POR', 21),
 ('FRA', 155)]

In [ ]:
#agrupa lista de equipos por pais
equiposOlimpicosRDD.map(lambda x: (x[2],x[1])).groupByKey().mapValues(list).take(2)

[('sigla', ['equipo']),
 ('AUT',
  ['30. Februar',
   'Austria',
   'Austria-1',
   'Austria-2',
   'Breslau',
   'Brigantia',
   'Donar III',
   'Evita VI',
   'May-Be 1960',
   '"R.-V. Germania; Leitmeritz"',
   'Surprise'])]

In [ ]:
#agrupa lista de equipos por pais especifico
equiposOlimpicosRDD.filter(lambda x: "ARG" in x).take(5)

[['4', 'Acturus', 'ARG'],
 ['37', 'Antares', 'ARG'],
 ['42', 'Arcturus', 'ARG'],
 ['43', 'Ardilla', 'ARG'],
 ['45', 'Argentina', 'ARG']]

In [ ]:
# rdd.count() #cuenta el numero de datos en el rdd
# rdd.countApprox(20) #contara durante la cantidad de minisegundos que le pongas
  


In [5]:
#sacamos data de dep olimpico
depOlimpicoRDD = sc.textFile(path+"deportista.csv") \
.map(lambda line : line.split(","))

depOlimpicoRDD2 = sc.textFile(path+"deportista2.csv") \
.map(lambda line : line.split(","))

#unimos ambos rdds
depOlimpicoRDD = depOlimpicoRDD.union(depOlimpicoRDD2)


In [ ]:
depOlimpicoRDD.top(2) #revisa los dos primeros deportistas

[['deportista_id', 'nombre', 'genero', 'edad', 'altura', 'peso', 'equipo_id'],
 ['99999', 'Alexander Grant Alick Rennie', '1', '32', '182', '71', '967']]

In [ ]:
#hacemos un join para ver el equipo de cada jugador
depOlimpicoRDD.map(lambda l: [l[-1],l[:-1]]) \
              .join(equiposOlimpicosRDD.map(lambda x: [x[0] , x[2]] )) \
              .takeSample(False,6, 25)#no valores repetidos , 6 valores a visualizar, semilla aleatoria

[('362', (['131505', 'Steven Woodburn', '1', '24', '185', '90'], 'FRA')),
 ('967', (['13626', 'Jill Brresen', '2', '22', '170', '57'], 'RSA')),
 ('482', (['44299', 'Gumundur Gumundsson', '1', '23', '174', '77'], 'ISL')),
 ('970', (['68062', 'Lee MinHui', '2', '28', '174', '65'], 'KOR')),
 ('794', (['92442', 'Luis Paz Zoldan', '1', '19', '187', '82'], 'PER')),
 ('413', (['26822', 'Jared Mark Deacon', '1', '24', '185', '77'], 'GBR'))]

In [6]:
#sacamos data de resultados
resultRDD = sc.textFile(path+"resultados.csv") \
.map(lambda line : line.split(","))


In [7]:
resultadoGanador = resultRDD.filter(lambda l : 'NA' not in l[1])
resultadoGanador.take(2)

[['resultado_id', 'medalla', 'deportista_id', 'juego_id', 'evento_id'],
 ['4', 'Gold', '4', '2', '4']]

In [26]:
#juntar rdd que acabamos de crear con el rdd de equipos y jugadores 
deportistaPaises = depOlimpicoRDD \
                   .map(lambda l : [l[-1],l[:-1]]) \
                   .join(equiposOlimpicosRDD.map(lambda x : [ x[0],x[2] ]))


deportistaPaises.join(resultadoGanador).take(6)

[('74',
  ((['65', 'Patimat Abakarova', '2', '21', '165', '49'], 'AZE'), 'Gold')),
 ('74', ((['129', 'Ruslan Abbasov', '1', '22', '181', '74'], 'AZE'), 'Gold')),
 ('74', ((['130', 'Tural Abbasov', '1', '18', '182', '76'], 'AZE'), 'Gold')),
 ('74', ((['131', 'Tran Abbasova', '2', '33', '159', '53'], 'AZE'), 'Gold')),
 ('74',
  ((['335', 'Abdulqdir Abdullayev', '1', '28', '188', '91'], 'AZE'), 'Gold')),
 ('74',
  ((['336', 'Arif Yadulla Abdullayev', '1', '27', '164', '60'], 'AZE'),
   'Gold'))]

In [30]:
#Operaciones numericas con RDDS

#valor de puntuaje de los paises 
valoresMedallas={'Gold':7,
'Silver':5,
'Bronze':4 }

paisesMedallas=deportistaPaises.join(resultadoGanador)

#todos las medallas de azerbayan
paisesMedallas = paisesMedallas \
                .map(lambda x: [x[1][0][-1], valoresMedallas[x[1][1]]] )


In [31]:
from operator import add # permite hacer sumas
conclusion = paisesMedallas.reduceByKey((add)).sortBy(lambda x : x[1],ascending=False)
conclusion.take(10)

[('CAN', 32538),
 ('ARG', 12520),
 ('HUN', 10860),
 ('MEX', 6124),
 ('RSA', 3788),
 ('BLR', 3580),
 ('LTU', 1535),
 ('MGL', 1460),
 ('USA', 1342),
 ('AZE', 1218)]

In [ ]:
spark.stop()